<a href="https://colab.research.google.com/github/gmauricio-toledo/NLP-MCD/blob/main/02-Preprocesamiento_b%C3%A1sico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Preprocesamiento de Texto</h1>

En esta notebook visualizaremos algunos conceptos del preprocesamiento de texto, particularmente:

* Tokenización
* Stemming
* Named Entity Recognition
* POS Tagging

Para esto, estudiaremos algunos ejemplos que serán procesados por las clases del módulo [spaCy](https://spacy.io/). No es una guía minuciosa del uso de este módulo, son solamente algunos ejemplos ilustrativos.

Verificar que la versión de spaCy sea la 3.*

In [ ]:
!python -m spacy info

In [ ]:
 import spacy

Ya que hemos importado spaCy, necesitamos cargar un modelo estadístico de lenguaje. SpaCy ofrece una variedad de modelos para diferentes idiomas. Estos modelos ayudan con la tokenization, etiquetado PoS (part-of-speech), NER (named entity recognition) y más.

Bajamos y cargamos el modelo estadístico de lenguaje **en_core_web_sm**, es el modelo más pequeño en inglés de spaCy y un buen punto de partida.

Documentación del modelo: https://spacy.io/models/en#en_core_web_sm<br>
Modelos disponibles: https://spacy.io/models<br>
Uso de los modelos: https://spacy.io/usage/models

🔵 Información adicional del modelo

**en_core_web_sm** fué entrenado en el corpus OntoNotes 5, el cual es un corpus anotado que contiene noticias, blogs, transcripciones, etc. Los documentos del corpus están anotados con información de como cada oración debería *parsearse* (parsing), part-of-speech de cada palabra, si cada palabra es una *named entity*, entre otras cosas.

https://catalog.ldc.upenn.edu/LDC2013T19



En principio no hay necesidad de bajarlo

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
nlp = spacy.load('en_core_web_sm')

Ya hemos cargado el modelo, la variable `nlp` hace referencia a una instancia de la clase [`Language`](https://spacy.io/api/language) que contiene metodos para varias tareas (tokenización, etc.) y un pipeline de procesamiento.

Usaremos este modelo de lenguaje para realizar algunas tareas de preprocesamiento de PLN.


In [ ]:
type(nlp)

# Tokenización


Al pasar cualquier texto a la instancia `nlp` obtenemos un objeto [`Doc`](https://spacy.io/api/doc) que contiene el texto tokenizado e información adicional para cada [token](https://spacy.io/api/token).

In [ ]:
# Sample sentence.
text = "He didn't want to pay $20 for this book."
doc = nlp(text)
print(doc,'\n',type(doc))

Veamos los tokens

In [ ]:
print([t.text for t in doc])

Observar que:
- "didn't" se separa en "did" y "n't".
- El símbolo de moneda y el número están separados.
- El punto final es también un token.

Una tokenización *naive*:

In [ ]:
text.split()

El objeto `Doc` puede ser indexado y *sliced* como si fuera una lista:

In [ ]:
print(doc[0])
print(type(doc[0]))

In [ ]:
print(doc[:3])
print(doc[-5:])

Podemos recuperar el texto original:

In [ ]:
print(doc.text)

Podemos tokenizar multiples oraciones y accesar a ellas individualmente usando la propiedad `sents` del objeto `Doc`.

In [ ]:
s = """Either the well was very deep, or she fell very slowly, for she
had plenty of time as she went down to look about her and to wonder what
was going to happen next. First, she tried to look down and make out what
she was coming to, but it was too dark to see anything; then she looked at
the sides of the well, and noticed that they were filled with cupboards and
book-shelves; here and there she saw maps and pictures hung upon pegs."""

doc = nlp(s)

lista_oraciones = [sent for sent in doc.sents]

print(lista_oraciones)
print(len(lista_oraciones))

Algunos casos de errores en la tokenización: https://github.com/explosion/spaCy/issues/3052



# Reconocimiento de entidades nombradas (NER)

Podemos buscar entidades nombradas en el texto usando las anotaciones de los tokens

In [ ]:
entities = list(doc.ents)

print(entities)
print(entities[0].label_)

In [ ]:
doc2 = nlp('''I have $20 worth of things to buy in New York city,
I can afford more things in Mexico or Latin America, certainly not the Starry Night painting though.
Lastly, I think I met Zeus at a McKenzie group consulting meeting in NYC last summer.''')

[(x,x.label_) for x in list(doc2.ents)]

Observa que el NER no agrupa tokens juntos, estas agrupaciones se hacen de forma separada en el atributo `ents`.

In [ ]:
[x.text for x in doc2]

Algunas etiquetas de las entidades son:

* PERSON People, including fictional
* NORP Nationalities or religious or political groups
* FACILITY Buildings, airports, highways, bridges, etc.
* ORGANIZATION Companies, agencies, institutions, etc.
* GPE Countries, cities, states
* LOCATION Non-GPE locations, mountain ranges, bodies of water
* PRODUCT Vehicles, weapons, foods, etc. (Not services)
* EVENT Named hurricanes, battles, wars, sports events, etc.
* WORK OF ART Titles of books, songs, etc.
* LAW Named documents made into laws
* LANGUAGE Any named language

In [ ]:
spacy.explain('GPE')

Spacy cuenta con algunos visualizadores para NER y otras tareas.

https://spacy.io/usage/visualizers

In [ ]:
from spacy import displacy

displacy.render(doc2, style='ent', jupyter=True)

Podemos obtener, además, los índices de cada entidad en el texto original.

In [ ]:
print([(ent.text, ent.label_, ent.start_char, ent.end_char) for ent in doc2.ents])

Para corpus domain-specific, un etiquetador NER puede ser afinado. En este ejemplo sería bueno que _The Martian_ fuera etiquetado como "FILM".

🔵 Esto puede aparecer más adelante en los ejercicios y en el curso.

In [ ]:
s = "Ridley Scott directed The Martian."
doc = nlp(s)
displacy.render(doc, style='ent', jupyter=True)

In [ ]:
spacy.explain('NORP')

## Ejercicios

⭕ ¿Qué utilidad puede tener la tokenización?

### 1. Exploración descriptiva de documentos mediante tokenización

En este primer ejemplo describiremos el contenido de varios documentos que tratan sobre baseball. Exploraremos los términos más frecuentes mediante la construcción de una nube de palabras. Haremos el ejercicios usando, y sin usar, tokenización.

In [ ]:
!pip install -qq wordcloud

In [ ]:
from sklearn.datasets import fetch_20newsgroups

docs_newsgroups = fetch_20newsgroups(subset='train',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=['rec.sport.baseball']
                                     )

lista_docs_20ng = docs_newsgroups.data
print(f"{len(lista_docs_20ng)} documentos")

Sin tokenizar:

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

wc = WordCloud(background_color="white")
wc_img = wc.generate(" ".join(lista_docs_20ng))

plt.figure(dpi=200)
plt.imshow(wc_img, interpolation='bilinear')
plt.axis("off")
plt.show()

Tokenizando:

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

docs_spacy = nlp(" ".join(lista_docs_20ng))
lista_tokens = [t.text for t in docs_spacy]

wc = WordCloud(background_color="white")
wc_img = wc.generate(" ".join(lista_tokens))

plt.figure(dpi=200)
plt.imshow(wc_img, interpolation='bilinear')
plt.axis("off")
plt.show()

Visualicemos los tokens junto con su frecuencia:

In [ ]:
from collections import Counter

dict(sorted(Counter(lista_tokens).items(), key=lambda x: x[1], reverse=True))

Ahora quitemos stopwords

In [ ]:
lista_tokens = [t.text for t in docs_spacy
                if t.text not in ["n't","."," ","'s","_","\n"]]

wc = WordCloud(background_color="white")
wc_img = wc.generate(" ".join(lista_tokens))
plt.figure(dpi=200)
plt.imshow(wc_img, interpolation='bilinear')
plt.axis("off")
plt.show()

### Imprime una lista de todas las entidades nombradas que aparecen en estos documentos

Al ser de baseball podrían aparecer nombres de jugadores y equipos, ¿el NER los atrapa?

### 2. Tokenizando con NLTK y NLP-Core

[NLTK](https://www.nltk.org/) (**N**atural **L**anguage **T**ool**K**it) es otro módulo para tareas de NLP.  Usando los tokenizadores `TreebankWordTokenizer` y `word_tokenize` de NLTK, tokeniza el parrafo un texto de tu elección y compara la tokenización con spacy. ¿Es la misma?

[Core-NLP](https://stanfordnlp.github.io/CoreNLP/) es una suite para tareas de NLP, está escrita en Java, por lo que no se puede usar directamente en Python. Puedes usar el [demo online](https://corenlp.run/). Tokeniza el mismo texto usando esta herramienta. ¿Qué más información puedes ver?


### 3. Tokenizando en español

* Prueba un modelo en español de scipy para tokenizar el siguiente texto. Inspecciona la lista de tokens. ¿Notas algún caso particular de interés?
* Revisa las entidades nombradas que reconoce, ¿le faltó alguna? ¿hay alguna que no sea correcta?

In [ ]:
texto = '''
Edward Benjamin Britten (Lowestoft, 22 de noviembre de 1913-Aldeburgh, 4 de diciembre de 1976), fue un compositor, director de orquesta y pianista británico. Fue una figura central de la música británica del siglo XX, con un abanico de obras que incluye ópera, otra música vocal y piezas orquestales y de cámara. Entre sus obras más conocidas figuran la ópera Peter Grimes (1945), el Réquiem de guerra (1962) y la pieza orquestal The Young Person's Guide to the Orchestra (1945).
'''

### 4. Usando los tokenizadores mencionados arriba, ¿la tokenización separa prefijos? Prueba en español e inglés.

### 5. Revisa los contenidos del curso [spaCy 101: Everything you need to know](https://spacy.io/usage/spacy-101)

Te ayudará a identificar qué tipo de cosas se pueden hacer con spacy para futuras referencias.

# Más preprocesamiento: Mayúsculas/Minúsculas, Stop Words Removal, Stemming y Lematización.


spaCy performs all these preprocessing steps (except stemming) behind the scenes for you. Inline with its non-destructive policy, the tokens aren't modified directly. Rather, each **Token** object has a number of attributes which can help you get views of your document with these pre-processing steps applied. The attributes a **Token** has can be found here:<br>
https://spacy.io/api/token#attributes
<br><br>
More information about spaCy's processing pipeline:<br>
https://spacy.io/usage/processing-pipelines

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

s = "Scientists have developed a new, more energy-efficient way for AI algorithms to process data. His model may become the basis for a new generation of AI that learns like we do."
doc = nlp(s)

## Mayúsculas/Minúsculas

Podemos escribir los tokens en minúsculas:

In [ ]:
print([t.lower_ for t in doc])

Esto nos da flexibilidad para realizar otras tareas, como no cambiar a minúscula si es el inicio de una oración.

In [ ]:
print([t.lower_ if not t.is_sent_start else t for t in doc])

## Stop Word Removal

Las stop words son palabras de una lista (stoplist) que se filtran antes o después del procesamiento de texto ya que se consideran insignificantes.

spaCy incluye una lista por default de stop words. Observa que cada token viene anotado con el atributo `is_stop`.

In [ ]:
print([t for t in doc if not t.is_stop])

Pdemos ver todas las stopwords. Podemos recuperarlas desde el módulo de spacy, o desde el modelo de lenguaje.

In [ ]:
from spacy.lang.en import stop_words

# --- Manera 1
stop_words = stop_words.STOP_WORDS
print(stop_words)

# --- Manera 2
stop_words_model = nlp.Defaults.stop_words
print(stop_words_model)
print(len(nlp.Defaults.stop_words))

Exploremos los signos de puntuación:

In [ ]:
from string import punctuation

punctuations = list(punctuation)
print(punctuations)
print(len(punctuations))

Además de quitar stop words también podemos quitar los signos de puntuación:

In [ ]:
print([t for t in doc if not t.is_stop and t.text not in punctuations])

## Lematización

La **lematización** es el proceso de reducir una palabra a su forma base (lema). Se utiliza para:

* Reducir la dimensionalidad del espacio de características, al mapear palabras relacionadas a un solo lema.
* Mejorar la precisión de los modelos de lenguaje, al tratar palabras con el mismo significado como una sola entidad.
* Facilitar la comparación y el análisis de textos, al estandarizar la forma de las palabras.

In [ ]:
[(t.text, t.lemma_) for t in doc]

Veamos los que cambiaron:

In [ ]:
[(t.text, t.lemma_) for t in doc if t.text != t.lemma_]

In [ ]:
import spacy

text = "He was running late, we were waiting, now we are here. Being together is the best"

nlp = spacy.load('en_core_web_sm')

doc = nlp(text)
[(t.text, t.lemma_) for t in doc]

## Ejercicios

### Repite la nube de palabras con los documentos del 20newsgroups

Produciremos nubes de palabras secuenciales:

* Quitando stopwords y signos de puntuación.
* Quitando stopwords y puntuación, además aplicando lematización.
* Quitando stopwords y puntuación, además aplicando stemming (spacy no lo soporta nativamente, busca opciones en NLTK).

¿Cuál consideras que captura de manera más clara la temática de los documentos?

### Repite la estrategia que hayas escogido como la mejor para analizar ahora otro conjunto de documentos.

Escoge documentos de otro tema del mismo corpus 20newsgroups. Aplica la misma estrategia anterior para producir dos nubes de palabras.


¿Consideras que ambas nubes de palabras reflejan las temáticas subyacentes de los documentos?

## Etiquetado Part-of-Speech


spaCy realiza, como parte de su pipeline, el etiquetado Part-of-Speech (POS).

In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')
s = "John watched an old movie at the cinema."
doc = nlp(s)

Las etiquetas POS se pueden ver en el atributo `pos_`

In [ ]:
[(t.text, t.pos_) for t in doc]

Con `spacy.explain` podemos obtener una descripción de las abreviaturas.

In [ ]:
spacy.explain('PROPN')

Con el atributo `tag_` podemos obtener información más detallada, acerca de cada token.

In [ ]:
[(t.text, t.tag_) for t in doc]

Descripción:

In [ ]:
print(spacy.explain('NNP'))
print(spacy.explain('VBD'))
print(spacy.explain('JJ'))

### Parsing

SpaCy también realiza el parsing como parte de su pipeline. Visualicemos un ejemplo.

In [ ]:
s = "She enrolled in the course at the university."
doc = nlp(s)

displacy.render(doc, style='dep', jupyter=True)

Para cada par de dependencias, spaCy muestra el nodo *hijo* (hacia donde apunta la flecha), el nodo *padre* (desde donde se apunta) y su relación (la etiqueta sobre la flecha).

Más información: https://spacy.io/api/annotation#dependency-parsing

Como siempre, podemos ver más detalles con `spacy.explain`.

In [ ]:
print(spacy.explain('nsubj'))
print(spacy.explain('dobj'))
print(spacy.explain('prep'))
print(spacy.explain('pobj'))

Podemos acceder a las etiquetas de dependenciaa través del atributo `dep_`.

In [ ]:
[(t.text, t.dep_) for t in doc]

Observa que 'enrolled' es el _ROOT_. A continuación podemos darnos una idea de cómo ocurre el etiquetado en el pipeline.

* nsubj (nominal subject) = SUJETO de la oración
* ROOT = El verbo principal de la oración
* obj (direct object) = objeto directo del verbo


In [ ]:
[(t.text, t.dep_, t.pos_, t.head.text) for t in doc]

Así podemos acceder a los nodos descendientes de un nodo en particular:

In [ ]:
for t in doc:
    if "obj" in t.dep_:
        print(' '.join([child.text for child in t.subtree]))

Analicemos los descendientes de un nodo `ROOT`

In [ ]:
for t in doc:
    if "ROOT" in t.dep_:
        # print(' '.join([child.text for child in t.subtree if child.dep_ != 'ROOT']))
        print(' '.join([child.text for child in t.subtree if child.pos_ != 'VERB']))

Podemos acceder y reconstruir la estructura gramatical de una oración de un documento:

In [ ]:
for sent in doc.sents:
    for token in sent:
        if token.dep_ == 'nsubj':
            subject = ' '.join([child.text for child in token.subtree])
            print(f"Sujeto: {subject}")
        elif token.dep_ == 'ROOT' and token.pos_ == 'VERB':
            print(f"Verbo: {token.text}")
            predicado = ' '.join([child.text for child in token.subtree if child.dep_!= 'nsubj' and child.dep_ != 'ROOT'])
            print(f"Objeto: {predicado}")